In [2]:
# IMPORT LIBRARIES

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import ConnectionPatch
from scipy.io import loadmat

import eelbrain
from eelbrain import UTS, NDVar
import mne
import re

In [4]:
# DATA LOCATIONS

#BASE_DIR = Path.cwd()
BASE_DIR = Path("~").expanduser() / 'Data' / 'cocoha' # <---- Uncomment this if you are working with files in data root directory

DATA_PREPROC = BASE_DIR / "data_preprocessed"
PREDICTOR_DIR = BASE_DIR / 'predictors'
TRF_DIR = BASE_DIR / 'TRFs'
PREDICTOR_DIR.mkdir(exist_ok=True)
TRF_DIR.mkdir(exist_ok=True)

STIMULUS_DIR = BASE_DIR / 'stimuli'
EEG_DIR = BASE_DIR / 'eeg'
EEG_DIR.mkdir(exist_ok=True)
SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d+', path.name)]
SUBJECTS = sorted(SUBJECTS, key=lambda x: int(re.search(r'S(\d+)', x).group(1)))

STIMULI = [stimulus.stem for stimulus in STIMULUS_DIR.glob("*.wav")]


In [5]:
# Load envelope TRFs for all subjects
trfs = []
r_values = []

for subject in SUBJECTS:
    attended_trf_path = TRF_DIR / subject / f'{subject}_attended_trf.pickle'
    unattended_trf_path = TRF_DIR / subject / f'{subject}_unattended_trf.pickle'

    if attended_trf_path.exists() and unattended_trf_path.exists():
        trf_attended = eelbrain.load.unpickle(attended_trf_path)
        trf_unattended = eelbrain.load.unpickle(unattended_trf_path)
        trfs.append((trf_attended, trf_unattended))
        
        r_values.append((trf_attended.r, trf_unattended.r))
        # Note that r does not take into account the model’s ability to predict the magnitude of the response, only its shape; for a measure that reflects both, consider using proportion_explained.

        print(f"Loaded TRFs for {subject}")
    else:
        print(f"TRF files for {subject} not found, skipping.")
        

print(f"Loaded TRFs for {len(trfs)} subjects")

Loaded TRFs for S1
Loaded TRFs for S2
Loaded TRFs for S3
Loaded TRFs for S4
Loaded TRFs for S5
Loaded TRFs for S6
Loaded TRFs for S7
Loaded TRFs for S8
Loaded TRFs for S9
Loaded TRFs for S10
Loaded TRFs for S11
Loaded TRFs for S12
Loaded TRFs for S13
Loaded TRFs for S14
Loaded TRFs for S15
Loaded TRFs for S16
Loaded TRFs for S17
Loaded TRFs for S18
Loaded TRFs for 18 subjects


In [ ]:
SUBJECT_NUMBER = 3

attended_trf_mean = trfs[SUBJECT_NUMBER -1 ][0].h_scaled.mean('sensor')
unattended_trf_mean = trfs[SUBJECT_NUMBER - 1][1].h_scaled.mean('sensor')

